---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !git clone https://github.com/SKTBrain/KoBERT.git

In [4]:
cd /content/drive/MyDrive/Kobert/KoBERT

/content/drive/MyDrive/Kobert/KoBERT


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [6]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# pip install transformers-interpret

In [8]:
import wandb

wandb.login()
wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: glad415. Use `wandb login --relogin` to force relogin


In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [10]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [11]:
#kobert
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [12]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

bertmodel = AutoModel.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


---
# 학습용 데이터셋 전처리
---

In [14]:
data = pd.read_csv('/content/drive/MyDrive/Kobert/data/final_emotional_dataset.csv')
del data['Unnamed: 0']
data = data.drop_duplicates(subset = 'content', keep = 'first')
data = data.dropna()
data = data.reset_index(drop = True)
data

,content,label
0,접근성 좋아요,1
1,친절함은 좋았으나 4인이 식사를 하는데 밑반찬까지 한그릇씩만 나와서 먹기에 불편 했...,0
2,고기는 맛있는데 서버들 몇명은 불친절하고 주문도 제대로 못받음,0
3,모든 면에서 강추합니다 주변에서 14년 살았는데 젤 좋습니다,1
4,사장님 직원분들 너무 친절하시구 음식도 맛있어요,1
...,...,...
102540,사람마다 틀리겠지만 불친절하네 말투하며 접수할맘도 안생기네 ㅎㅎ 따른데 갈란다,0
102541,이집 소개해서 욕먹은적없는 곱창 좀 먹어본분들이 가시면 좋습니다 양마니와 오발탄을 ...,1
102542,메뉴 구성도 좋고 맛있고 직원분들도 친절하고 깔끔하고 사장님 친절하시고 퍼펙트합니다,1
102543,먹어본 공차중에 버블 제일 딱딱하고 적음 최악,0


In [15]:
data['positive'] = None
data.columns = ['content','positive','negative']
data

,content,positive,negative
0,접근성 좋아요,1,None
1,친절함은 좋았으나 4인이 식사를 하는데 밑반찬까지 한그릇씩만 나와서 먹기에 불편 했...,0,None
2,고기는 맛있는데 서버들 몇명은 불친절하고 주문도 제대로 못받음,0,None
3,모든 면에서 강추합니다 주변에서 14년 살았는데 젤 좋습니다,1,None
4,사장님 직원분들 너무 친절하시구 음식도 맛있어요,1,None
...,...,...,...
102540,사람마다 틀리겠지만 불친절하네 말투하며 접수할맘도 안생기네 ㅎㅎ 따른데 갈란다,0,None
102541,이집 소개해서 욕먹은적없는 곱창 좀 먹어본분들이 가시면 좋습니다 양마니와 오발탄을 ...,1,None
102542,메뉴 구성도 좋고 맛있고 직원분들도 친절하고 깔끔하고 사장님 친절하시고 퍼펙트합니다,1,None
102543,먹어본 공차중에 버블 제일 딱딱하고 적음 최악,0,None


In [16]:
for i in range(len(data)):
  if data['positive'][i] == 1:
    data['negative'][i] = 0
  else :
    data['negative'][i] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
data

,content,positive,negative
0,접근성 좋아요,1,0
1,친절함은 좋았으나 4인이 식사를 하는데 밑반찬까지 한그릇씩만 나와서 먹기에 불편 했...,0,1
2,고기는 맛있는데 서버들 몇명은 불친절하고 주문도 제대로 못받음,0,1
3,모든 면에서 강추합니다 주변에서 14년 살았는데 젤 좋습니다,1,0
4,사장님 직원분들 너무 친절하시구 음식도 맛있어요,1,0
...,...,...,...
102540,사람마다 틀리겠지만 불친절하네 말투하며 접수할맘도 안생기네 ㅎㅎ 따른데 갈란다,0,1
102541,이집 소개해서 욕먹은적없는 곱창 좀 먹어본분들이 가시면 좋습니다 양마니와 오발탄을 ...,1,0
102542,메뉴 구성도 좋고 맛있고 직원분들도 친절하고 깔끔하고 사장님 친절하시고 퍼펙트합니다,1,0
102543,먹어본 공차중에 버블 제일 딱딱하고 적음 최악,0,1


In [18]:
import numpy as np 

total_data_text = list(data['content'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  73.12752450143839
문장길이 최대 :  2025
문장길이 표준편차 :  94.56406201708111
pad_sequences maxlen :  262
전체 문장의 0.9617046174850066%가 maxlen 설정값 이내에 포함됩니다. 


In [19]:
train_data, test_data = train_test_split(data, test_size = 0.2 ,random_state= 632)

In [20]:
print(train_data.shape)
print(test_data.shape)

(82036, 3)
(20509, 3)


---
# 모델
---

In [21]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, bert_tokenizer, max_len):
    
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=True, padding="max_length",
                                     max_length = max_len, return_tensors = 'pt') for i in dataset['content']]
    self.max_len = max_len
    self.positive = [np.int64(i) for i in dataset['positive']]
    self.negative = [np.int64(i) for i in dataset['negative']]
  
  def __getitem__(self,i):
    self.input_ids = self.sentences[i]['input_ids']
    self.attention_mask_token = self.sentences[i]['attention_mask']
    self.token_type_ids = self.sentences[i]['token_type_ids']

    return self.input_ids, self.attention_mask_token, self.token_type_ids, self.positive[i], self.negative[i]

  def __len__(self):
    return (len(self.positive))

In [22]:
# 하이퍼파라미터 세팅
max_len = 262
batch_size = 16
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 600
learning_rate =  1e-5

In [23]:
data_train = BERTDataset(train_data, tokenizer, max_len)
data_test = BERTDataset(test_data, tokenizer, max_len)

In [24]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)

In [25]:
# dataloader test 
for a,b,c,d,e in train_dataloader:
  print(a)
  print(b)
  print(c)
  print(d)
  print(e)
  print('='*100)
  print('input_ids: ',a.shape)
  print('attention_mask: ',b.shape)
  print('token_type_ids: ',c.shape)
  print('positive_label: ',d.shape)
  print('negative_label: ',d.shape)
  break


tensor([[[    2,  4184,  2098,  ...,     0,     0,     0]],

        [[    2,  7966,  2289,  ...,     0,     0,     0]],

        [[    2, 17482,  3633,  ...,     0,     0,     0]],

        ...,

        [[    2, 13180,  2601,  ...,     0,     0,     0]],

        [[    2, 29867,  2225,  ...,     0,     0,     0]],

        [[    2,  4814, 13738,  ...,     0,     0,     0]]])
tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])
tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])
tensor([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0])
tensor([0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1])
input_ids:  torch.Size([16, 1, 262

In [26]:
wandb.init(project = 'kobert_emotional',
                 entity="glad415",
                 config = {
                     "hidden_size" : 768,
                     "max_len" : max_len,
                     "batch_size" : batch_size,
                     "learning_rate" : learning_rate,
                     "epochs" : num_epochs,
                     "optimizer" : 'SGD',
                     "loss" : "BCEloss",
                 })

In [27]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=1,   
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.positive = nn.Linear(hidden_size , num_classes)
        self.negative = nn.Linear(hidden_size , num_classes)

        self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.bert(input_ids = input_ids.long(), token_type_ids = token_type_ids.long(), attention_mask = attention_mask.to(device))
        out = self.dropout(out.pooler_output)

        positive_out = self.positive(out)
        negative_out = self.negative(out)

        return self.sigmoid(positive_out), self.sigmoid(negative_out)
    
    ############################# Frezee #############################
    # def freeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = False
    
    # def unfreeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = True
    ##################################################################

In [28]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
loss_fn = nn.BCELoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [29]:
for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, positive_label, negative_label) in enumerate(tqdm_notebook(train_dataloader)):
        
        optimizer.zero_grad()
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        positive_label = positive_label.float().to(device)
        positive_label = positive_label.unsqueeze(1)

        negative_label = negative_label.float().to(device)
        negative_label = negative_label.unsqueeze(1)


        positive_out, negative_out = model(input_ids, attention_mask_token, token_type_ids)

        # Loss
        ################################################################################################
        positive_loss = loss_fn(positive_out, positive_label)
        negative_loss = loss_fn(negative_out, negative_label)

        total_loss = (positive_loss + negative_loss)
        total_loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # Accuracy
        ################################################################################################
        positive_preds = (positive_out >= 0.5).float()
        negative_preds = (negative_out >= 0.5).float()

        positive_accuracy = (positive_preds == positive_label).cpu().numpy().mean()
        negative_accuracy = (negative_preds == negative_label).cpu().numpy().mean()

        train_acc += (positive_accuracy + negative_accuracy) / 2
        ################################################################################################

        # F1_Score
        ################################################################################################
        positive_f1 = f1_score(positive_preds.cpu().data, positive_label.cpu(), average = 'macro')
        negative_f1 = f1_score(negative_preds.cpu().data, negative_label.cpu(), average = 'macro')
        
        total_f1 +=  (positive_f1 + negative_f1) / 2
        ################################################################################################

        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {total_loss.data.cpu().numpy():.3f} train acc {train_acc / (batch_id+1):.3f}")
            print(f'positive_accuracy {positive_accuracy:.3f}  | negative_accuracy {negative_accuracy:.3f} | f1_score:{total_f1 / (batch_id+1):.3f}')
            print('#'*80)

            # WandB
            #############################################################################################################################
            wandb.log({'interval_Loss': total_loss.data.cpu().numpy(),
                        'interval_positive_Accuracy' :positive_accuracy, 'interval_negative_Accuracy' : negative_accuracy,
                        'interval_positive_F1' : positive_f1, 'interval_negative_F1' : negative_f1})
            #############################################################################################################################

    print(f"epoch {e+1} epoch_Train_Accuracy {train_acc / (batch_id+1):.3f}, epoch_F1_score {total_f1 / (batch_id+1):.3f}")
    
    # WandB
    #############################################################################################################################
    wandb.log({'epoch_Train_Accuracy' : train_acc / (batch_id+1), 'epoch_F1_score' : total_f1 / (batch_id+1)})
    #############################################################################################################################

    model.eval()
    with torch.no_grad():
      test_f1 = 0.0
      test_acc = 0.0

      for batch_id, (input_ids, attention_mask_token, token_type_ids, positive_label, negative_label) in enumerate(tqdm_notebook(test_dataloader)):
          
          optimizer.zero_grad()
          
          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          positive_label = positive_label.float().to(device)
          positive_label = positive_label.unsqueeze(1)

          negative_label = negative_label.float().to(device)
          negative_label = negative_label.unsqueeze(1)

          positive_out, negative_out = model(input_ids, attention_mask_token, token_type_ids)

          # Test_Accuracy
          ################################################################################################    
          positive_preds = (positive_out >= 0.5).float()
          negative_preds = (negative_out >= 0.5).float()

          positive_accuracy = (positive_preds == positive_label).cpu().numpy().mean()
          negative_accuracy = (negative_preds == negative_label).cpu().numpy().mean()

          test_acc += (positive_accuracy + negative_accuracy) / 2
          ################################################################################################ 

          # Test_F1_Score
          ################################################################################################
          positive_f1 = f1_score(positive_preds.cpu().data, positive_label.cpu(), average = 'macro')
          negative_f1 = f1_score(negative_preds.cpu().data, negative_label.cpu(), average = 'macro')
          
          test_f1 +=  (positive_f1 + negative_f1) / 2
          ################################################################################################

      print('-'*40,'Test_Value','-'*40)
      print(f"test_Accuracy {test_acc / (batch_id+1):.3f}, test_F1_score {test_f1/ (batch_id+1):.3f}")
      print('-'*92)

      wandb.log({'Test_acc' : test_acc / (batch_id+1), 'Test_F1_Score' : test_f1/ (batch_id+1)})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/5127 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.391 train acc 0.500
positive_accuracy 0.500  | negative_accuracy 0.500 | f1_score:0.479
################################################################################
epoch 1 batch id 601 loss 1.411 train acc 0.487
positive_accuracy 0.438  | negative_accuracy 0.562 | f1_score:0.457
################################################################################
epoch 1 batch id 1201 loss 1.475 train acc 0.499
positive_accuracy 0.500  | negative_accuracy 0.562 | f1_score:0.472
################################################################################
epoch 1 batch id 1801 loss 1.408 train acc 0.518
positive_accuracy 0.500  | negative_accuracy 0.688 | f1_score:0.494
################################################################################
epoch 1 batch id 2401 loss 1.244 train acc 0.539
positive_accuracy 0.875  | negative_accuracy 0.500 | f1_score:0.517
################################################################################
epoch 1 batch 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1281 [00:00<?, ?it/s]

...

  0%|          | 0/5127 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.561 train acc 0.938
positive_accuracy 0.938  | negative_accuracy 0.938 | f1_score:0.935
################################################################################
epoch 3 batch id 601 loss 0.499 train acc 0.918
positive_accuracy 0.938  | negative_accuracy 1.000 | f1_score:0.912
################################################################################
epoch 3 batch id 1201 loss 0.532 train acc 0.916
positive_accuracy 0.938  | negative_accuracy 1.000 | f1_score:0.911
################################################################################
epoch 3 batch id 1801 loss 0.496 train acc 0.915
positive_accuracy 1.000  | negative_accuracy 1.000 | f1_score:0.910
################################################################################
epoch 3 batch id 2401 loss 0.836 train acc 0.917
positive_accuracy 0.875  | negative_accuracy 0.875 | f1_score:0.911
################################################################################
epoch 3 batch 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1281 [00:00<?, ?it/s]

---------------------------------------- Test_Value ----------------------------------------
test_Accuracy 0.918, test_F1_score 0.912
--------------------------------------------------------------------------------------------


In [30]:
wandb.finish()

Test_F1_Score,▁██
Test_acc,▁██
epoch_F1_score,▁██
epoch_Train_Accuracy,▁██
interval_Loss,▇███▆▆▆▄▆▄▄▄▂▂▃▂▆▂▁▁▁▁▃▁▁▂▂
interval_negative_Accuracy,▁▂▂▄▁▂▅▅▅▆▅▆▇█▇▇▄▆▇███▆▇▇▇▇
interval_negative_F1,▁▁▂▄▁▂▅▄▅▆▅▆▇█▇▇▄▆▇███▆▇▇▇▇
interval_positive_Accuracy,▂▁▂▂▆▆▅█▅█▆▆▇█▇▇▆▆▇▇▇█▆▇▆▇▇
interval_positive_F1,▂▁▁▂▆▆▅█▅█▅▆▇█▇▇▆▆▇▇▇█▆▇▆▇▇
Test_F1_Score,0.91242
Test_acc,0.91796


In [31]:
!nvidia-smi

Thu Sep  8 04:10:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    44W / 300W |   6881MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
torch.save(model, '/content/drive/MyDrive/Kobert/model/koBERT_emotional_0907.pt')

In [ ]:
def predict(predict_sentence):

    dataset_another = pd.DataFrame({'content': predict_sentence, 'positive' : 0 , 'negative' : 0}, index = [0])

    another_test = BERTDataset(dataset_another,tokenizer, max_len)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)
    
    model.eval()

    for batch_id, (input_ids, attention_mask_token, token_type_ids, positive_label, negative_label) in enumerate(test_dataloader):
          
          optimizer.zero_grad()
          
          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          positive_label = positive_label.float().to(device)
          positive_label = positive_label.unsqueeze(1)

          negative_label = negative_label.float().to(device)
          negative_label = negative_label.unsqueeze(1)


          positive_out, negative_out = model(input_ids, attention_mask_token, token_type_ids)

          for positive, negative in zip(positive_out, negative_out):

              logits_positive = positive.detach().cpu().numpy()
              logits_negative = negative.detach().cpu().numpy()

              print('긍정 점수:',logits_positive)
              print('부정 점수:',logits_negative)
          

---
# 결과값 테스트
---

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
predict('맛이 없다')

긍정 점수: [0.15513323]
부정 점수: [0.81615496]


In [48]:
predict('서비스가 좋아서 다시 가고 싶어요')

긍정 점수: [0.86998546]
부정 점수: [0.11868027]


In [ ]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

In [49]:
predict('음식은 겉바속촉 매장은 깔끔')

긍정 점수: [0.77700907]
부정 점수: [0.26341477]


In [50]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

긍정 점수: [0.8968712]
부정 점수: [0.10565656]


In [40]:
predict('맛 양 배달 모두 만족합니다')

긍정 점수: [0.8592038]
부정 점수: [0.14126104]


In [41]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

긍정 점수: [0.8708801]
부정 점수: [0.143801]


In [42]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

긍정 점수: [0.15729219]
부정 점수: [0.8380406]


In [43]:
predict('사업 번창하세요')

긍정 점수: [0.7747145]
부정 점수: [0.24911936]


In [51]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고 ㅜㅜ')

긍정 점수: [0.19722615]
부정 점수: [0.83228856]


In [45]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

긍정 점수: [0.8874876]
부정 점수: [0.13265207]


In [54]:
predict('다른집 들렸다 온것 같아요')

긍정 점수: [0.26591292]
부정 점수: [0.67043865]


In [52]:
predict('배달은 빠른데 음식은 맛없어 ')

긍정 점수: [0.12254836]
부정 점수: [0.852573]
